In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.util import Dynamic
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize

from holoviews.streams import *
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

# set pandas dataframe float precision 
pd.set_option('display.precision',2)


In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
import utils as u
import hv_utils as  hvu


In [ ]:
mro = u.get_mro

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom']),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover']),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),

    opts.RGB(active_tools=['wheel_zoom']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


In [ ]:
H,W = 500,500

---
## Load Datasets

In [ ]:
# Southern Africa Dataset
data_dir = Path.home()/'data'
fpath_sa = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
)
fpath_ea = str(
    data_dir/'mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
)
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

---
## Back to FLDAS
Modified: Jun 25, 2019

Combining holoviews objects with Bokeh models for custome interactive visualization


In [ ]:
# Set extra style opts (in addition to default from above)
W_IMG = 500; H_IMG = 500
W_PLOT = 300; H_PLOT = 300

In [ ]:
scatter_opts = dict(width=W_PLOT, height=H_PLOT,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W_PLOT, height=H_PLOT,
                  framewise=True)
img_opts = dict(width=W_IMG, height=H_IMG,
                axiswise=True, 
                framewise=False,
                tools=['hover', 'tap'],
                colorbar=True
               )
wmts_opts = dict(width=W_IMG, height=H_IMG)

tbl_opts = dict(width = W_PLOT)

# datashader opts
ds_opts = dict(width=W_IMG, height=H_IMG,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

## Put the holoviews objects together 
- to form a parameterized class object

Modified: Jun 27, 2019


In [ ]:
trange = list(map(pd.Timestamp, hvd_ea.range('time')))

class ZonalExplorer(param.Parameterized):
    
    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[10], objects=varnames)
    
    
    ################################################################################
    ## Initialize instance
    ################################################################################
    def __init__(self, **params):
        super().__init__(**params)
        self._update()
        
#         self._init_streams()
#         self._set_dyn_main()
#         self._set_dyn_funcs()
        
    @param.depends('region', 'varname', watch=True)
    def _update(self):
        """
        (Re)Set main attributes: region data, time_values, empty_tplot
        in reaction to region and variable name parameter change from the user
        """
        print("Update is called")
        self.xrd = xrd_ea if self.region == 'EA' else xrd_sa
        self.time_values = u.to_datetime(self.xrd.get_index('time'))
        self.empty_tplot = self.get_empty_tplot()
        self.vlines = self.get_dmap_vlines()
        self.dmap_timg = self.get_dmap_timg()
        self.realize_dmaps()
        
        # add polygons and reset boxedit stream
        self.polys = gv.Polygons([], crs=ccrs.PlateCarree())
        self.box_stream = BoxEdit(source=self.polys)
        self.dmap_timg = basemap * self.dmap_timg * self.polys
        self.dmap_roi_curves = hv.DynamicMap(self.roi_curves, streams=[self.box_stream])
        print("Finished updating xrd, time_values, empty_tplot, dmap_vlines)")
        
    def _set_streams(self):
        # Set BoxEdit stream
        self.polys = gv.Polygons([], crs=ccrs.PlateCarree())
        self.box_stream = BoxEdit(source=self.polys)

    ################################################################################
    ## Basic Building Blocks -- generic functions
    ################################################################################  
    def get_empty_tplot(self):
        """
        Get an 'empty' time curve (hv.Curve) with the xaxis set as dt.datetime values
        correpsonding to self.time_values. This can serve as the background plot for 
        hv.VLine and roi_curves for boxedit selection functionality
        
        Returns:
        - hv.Curve
        """
        print("get_empty_tplot is called")
        dummy_df = pd.DataFrame({'time': self.time_values, 
                                 self.varname: np.zeros(len(self.time_values))})
        empty_tplot= hv.Curve(dummy_df, 'time', self.varname).opts(line_alpha=0., 
                                                                  bgcolor=(0,0,0,0))
        return empty_tplot
    
    def get_dmap_vlines(self, **vline_opts):
        """
        Construct a hv.Holomap of VLines at each time point in self.time_values
        
        Returns:
        hv.Holomap of hv.VLines with kdims of time
        """
        vlines = hv.DynamicMap(lambda time: self.empty_tplot * hv.VLine(time), kdims='time') 
        return vlines#.opts(**vline_opts)
        
    def get_dmap_timg(self):
        dmap_timg = datashade(hv.DynamicMap(self.get_timg, kdims='time'),
                      **ds_opts)
        return dmap_timg

    def get_timg(self, time):
        print("Getting a new timg...")
        print(f'-- Time: {time}, Region: {self.region}, Variable: {self.varname}')
        
        data = self.xrd[self.varname].sel(time=time)
        return gv.Image(data, ['X','Y'], self.varname, crs=ccrs.PlateCarree())
    
    def realize_dmaps(self):
        """
        Set the values for dimensions of dynamic maps
        Inplace operation on any dynamic map type attributes
        """
        self.vlines = self.vlines.redim.values(time=self.time_values)
        self.dmap_timg = self.dmap_timg.redim.values(time=self.time_values)

        
    ################################################################################
    ## Link streams Callbacks
    ################################################################################
    def roi_curves(self, data):
        if not data or not any(len(d) for d in data.values()):
            return hv.NdOverlay({0: self.empty_tplot})
        curves = {}
        data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
        for i, (x0,x1,y0,y1) in enumerate(data):
            selection = hv.Dataset( self.xrd[self.varname].sel(X=slice(x0,x1), Y=slice(y0,y1)), 
                                   kdims=['X','Y','time'])
            curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', self.varname)
        return hv.NdOverlay(curves, label='roi_curves').opts(
            opts.Curve(framewise=True))
    
    
    ################################################################################
    ## Print states
    ################################################################################ 
    def pp(self):
        self.print_param_values()
        
    ################################################################################
    ## Layouts and Views
    ################################################################################
    @param.depends('region', 'varname', watch=True)
    def panel_roi(self):
        w_img = int(W_IMG*1.8)
        h_img = int(W_IMG*1.8)
        w_plot = w_img
        h_plot = H_PLOT
        layout = (self.dmap_timg + self.dmap_roi_curves*self.vlines).opts(
                opts.WMTS(width=w_img, height=h_img),
                opts.Image(width=w_img, height=h_img),
                opts.RGB(width=w_img,height=h_img),
                opts.Curve(width=w_plot, #height=h_plot,
                           framewise=True),
                opts.VLine(color='black'),
                opts.Polygons(fill_alpha=0.2, line_color='white', fill_color='orange'), 
                ).cols(1)
        p = pn.panel(layout)
        roi_twidget =  p[1][0]
        roi_main = p[0]
        # Customize the components and final arrangement
        
        return pn.Row(
            pn.Pane(roi_twidget, width=300),
            roi_main)
    
    
    
    

In [ ]:
ze = ZonalExplorer()

In [ ]:
## initialization test
# ex.pp()
ze.panel_roi()

In [ ]:
# ex.vlines
# ex.empty_tplot
# ex.dmap_timg + ex.vlines

In [ ]:
#test
# pn.Column(
#     pn.panel(ze.param),
#     pn.panel(ze.panel_roi)
# )#.servable()
# Thank goodness! this works:)

## Finally putting everything together
Modified: Jun 27, 2019


In [ ]:
# Set extra style opts (in addition to default from above)
W_IMG = 400; H_IMG = 400
W_PLOT = 800; H_PLOT = 300

In [ ]:
scatter_opts = dict(width=W_PLOT, height=H_PLOT,
                    tools=['hover', 'tap'], 
                    framewise = True)
curve_opts = dict(width=W_PLOT, height=H_PLOT,
                  framewise=True)
img_opts = dict(width=W_IMG, height=H_IMG,
                axiswise=True, 
                framewise=False,
                tools=['hover', 'tap'],
                colorbar=True
               )

tbl_opts = dict(width = W_PLOT)

wmts_opts = dict(width=W_IMG, height=H_IMG)
geoline_opts = opts.Shape(line_color='black', line_dash='dashed')
inverted_curve_opts = opts.Curve('InvertedAxes', yaxis='right', invert_axes=True, show_title=False)
# curve_y_opts(opts.Curve(yaxis='right', invert_axes=True

# datashader opts
ds_opts = dict(width=W_IMG, height=H_IMG,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

In [ ]:
import datetime as dt


trange = list(map(pd.Timestamp, hvd_ea.range('time')))
basemap.opts(**wmts_opts)

class FLDASExplorer(param.Parameterized):

    ################################################################################
    ## Parameters
    ################################################################################
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[0],objects=varnames)
    time = param.Date(trange[0], bounds=trange)
#     time_slider= param.Date(trange[0], bounds=trange)


    ################################################################################
    ## Initialize instance
    ################################################################################
    def __init__(self):
        super().__init__()
        self._init_streams()
        self._init_debug_box()
        self._set_dyn_main()
        self._set_dyn_funcs()
        
        
    ################################################################################
    ## Initialize Link streams inbetween static components
    ################################################################################
    @param.depends('region', 'varname', 'time', watch=True)
    def _set_dyn_main(self):
        print("Setting main plot dynamically...")
        print('\tTime: ', self.time, type(self.time))
        dmap_img = hv.DynamicMap(self.get_main_img)
        dyn_img = datashade(Dynamic(dmap_img),**ds_opts)

        self.dyn_main = basemap * dyn_img
    
        # must relink the streams 
        self.dyn_img = dyn_img
        self.tap_stream.source = self.dyn_img
    
    def _set_dyn_funcs(self):
        # todo: add set_dyn_main
        
        self.dyn_tseries = hv.DynamicMap(self.cb_tseries, streams=[self.tap_stream] )
        self.dyn_tstats = hv.DynamicMap(self.cb_tstats, streams=[self.tap_stream])
        self.dyn_second_plot = basemap* datashade(hv.DynamicMap(self.cb_second_plot, streams=[self.sel_stream]),
                              **ds_opts)
        self.dyn_third_plot = basemap * datashade(hv.DynamicMap)

        
        
    def _init_streams(self):
        self.tap_stream = Tap(name='tap_latlon', x=35.0, y=0.0)#, source=self.main_plot)
        self.sel_stream = Selection1D()
        self.dummy = Tap(name='dummy', x=0, y=0)
        self.streams = [self.tap_stream, self.sel_stream]
        print("Created initial tap and selection1d stream instances...")

    def _init_debug_box(self):
        print("Initialing a debug box...")
        self.debug_box = hv.Div("""
        <h1 style='color:blue;border:1px solid blue'>Debug Box</h1>
        """)
        self.debug_dmap = hv.DynamicMap(self.get_updated_debug_box, 
                                       streams=self.streams)
        print('Initiated debug_dmap with current streams to debug_box')
        
    def get_updated_debug_box(self, *args, **kwargs):
        hvu.append_to_div(self.debug_box, str(args))
        hvu.append_to_div(self.debug_box, str(kwargs))
        return self.debug_box
    
    
    ################################################################################
    ## Dynamic Building Blocks (hv.DynamicMaps)
    ################################################################################        
    @param.depends('region', 'varname', 'time')#, watch=True)
    def get_main_img(self):
        print("Getting a new main img...")
        print(f'\tTime: {self.time}, Region: {self.region}')
        xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = hvu.get_img(xrd, self.varname, self.time)
        # debug
        
        return img.relabel(self.varname).opts(**img_opts)

        
    ################################################################################
    ## Link streams Callbacks
    ################################################################################
    @param.depends('region', 'varname')
    def cb_tseries(self, x, y, method='nearest'):
        """
        Affects:
        - Until the first time this callback  called by the instance, there is no
        self.tscatter attribute 
        - Everytime it is called, self.tscatter is updated to the new scatter plot 
        object that resides in the caller hv.DynamicMap.plot instance
            - The reason to maintain which self.tscatter we use is to 
        """
        print(f'cb_tseries is called. x,y: {x,y, self.region,self.varname}"')
        xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = xrd[self.varname].sel(X=x, Y=y, method=method)
        tseries_label = f"Time Series for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "

        # Time series as Scatter and Curve
        print('\tself.scatter is updated')
        self.tscatter = hv.Scatter(tseries, label=self.varname)
        curve = hv.Curve(tseries)

        # Important! Connect the self.sel_stream to this new tscatter
        print("\tself.sel_stream's source is changed to the new self.tscatter object")
        self.sel_stream.source = self.tscatter

        # Add HLine at mean over time
        mean = hvu.extract_item(tseries.mean())
        mean_line = hv.HLine(mean, label='tseries_mean')
        t_midpoint = pd.Timestamp(hvu.extract_item(tseries.coords['time'][len(tseries)//2]))
        mean_label = hv.Labels([(t_midpoint, mean, f'mean: {mean:.3f}')])

        # Put together to an Overlay
        overlay = (
            curve.opts(alpha=0.5, line_width=2, **curve_opts) 
            * self.tscatter.opts(padding=0.2, size=10, **scatter_opts) 
            * mean_line.opts(color='black', alpha=0.5, line_width=1)
            * mean_label.opts(text_font_size='8pt',text_alpha=0.5)
        ) 
        overlay = overlay.relabel(tseries_label)
#         self.overlay = overlay

        return overlay #vs. self.overlay
    
    @param.depends('region', 'varname')
    def cb_tstats(self, x, y, 
                  method='nearest',show_as_timestamp=True, decimals=3):
        print(f"cb_tstats is called: {x, y, self.region, self.varname}")
        xrd = xrd_ea if self.region == 'EA' else xrd_sa  
        tseries = xrd[self.varname].sel(X=x, Y=y, method=method)
        df = hvu.get_stats(tseries, 
                           show_as_timestamp=show_as_timestamp, decimals=decimals)

        # Add metadata on selected latlon point
    #     df['point_idx'] = index[0]#
        df['lat'] = y
        df['lon'] = x

        cols = df.columns.to_list()
        cols = cols[-2:] + cols[:-2]
        df = df[cols]
        label = f"Time Series Stats for {self.varname} at Lon,Lat = ({x:.2f},{y:.2f}) "
        print("\tUpdated self.tstats_table")
        self.tstats_table = hv.Table(df, label=label).opts(**tbl_opts)
        return self.tstats_table
    
    @param.depends('region', 'varname')
    def cb_second_plot(self, index):
        print('Callback from the scatter plot is called. Selected indices: ', index)
        if not index:
            index = [0] #todo: current tindex
        tidx = index[0]
        
        xrd = xrd_ea if self.region == 'EA' else xrd_sa 
        img = hvu.get_img(xrd, self.varname, tidx)
        label = f"{self.varname} on {self.time.strftime('%Y/%m/%d')}"
        return img
    
    @param.depends('region', 'varname')
    def cb_third_plot(self, x,y, method='nearest'):
        print('cb_third_plot is called. x,y clicked: ', x, y)
        
        xrd = xrd_ea if self.region == 'EA' else xrd_sa 
        img = hvu.get_img(xrd, self.varname, tidx)
        label = f"{self.varname} on {self.time.strftime('%Y/%m/%d')}"
        return img
    

    ################################################################################
    ## Build views
    ################################################################################
    @param.depends('region', 'varname', 'time')
    def view(self):
        row1 = (self.dyn_main + self.dyn_second_plot).opts(
            opts.Image(**img_opts))
        row2 = self.dyn_tseries.opts(
            opts.Scatter(**scatter_opts),
            opts.Curve(**curve_opts))
        row3 = self.dyn_tstats.opts(opts.Table(**tbl_opts) )

        return pn.Column(row1, row2, row3)

## Instruction Template

In [ ]:
title = """#MINT Interactive Visualization for Spatiotemporal Data"""
instruction = """
This is a demo of an interactive visualization for a large set of spatiotemporal data. 
Here we use the FLDAS NOAH01 daily data in Southern Africa (SA) and Eastern Africa (EA) for the month of April in 2019.
This demo supports a **correct** visualization of a **large** dataset without falling into common visualization [mistakes](#) 
such as overplotting and deceptive colormapping via [Datashader]() and [Colorcet]().


## Instructions

1. Dataset selection

    - dropdown boxes: choose `region`, `variable`, `time`  of interest
    - To specify `time`, a user can either type a specific time to the `Time` input box
    or use the time slider to navigate through multiple time points

2. All maps are responsive to user inputs from the mouse or trackpad.

    - pan or zoom: navigate the map
    - mouse hover or click: get its value or see through multiple time points at a specific (Lat, Lon)
    - drag or box-select: compute zonal statistics in the selected area

3. The time-series plot also supports interactivity

    - pan, zoom, mouse-hover as above
    - click on a point: allows to see the data at the current timepoint over the entire region

"""

viz_title = """#FLDAS Explorer POC"""
tseries_title = """##Time Series Analysis"""
zonal_title = """##FLDAS ROI Explorer"""

zonal_instruction = """
The dashboard additionally supports users to explore multiple ROIs across time. In the following map, we can
first choose the dataset by specifying the region and data varialbe using the dropdown boxes. The timeslider
allows you to navigate the dataset over all available time points. 

Plot
To define an ROI, select the 'Box edit' tool and double click to start defining the ROI and double click to 
finish placing the ROI

Upon the selection of ROI, the measurements aggregated over the ROI at each time point is ploted as a time 
series.  This is similar to the time seires of point measurements at a specific (lat,lon) above, but the 
measurements are aggregated over space. 

You can select multiple ROIs and the time series plots will be updated as more ROIs are added. 

Additionally, you can use the time slider to see both the measurements for the entire region and the aggregated
data at specific time.
"""


contact = """
## Suggestions?
We would like to know what kind of functionalities users are most interested in.  
Please drop a line for requests on this google [doc](https://docs.google.com/spreadsheets/d/14NtyaHcdWnwerDDANt8SOCIIZl_rQzGKzl9GvH0_uk8/edit#gid=0), and we'll start working on adding supports for popular requests!

## Questions?
Please feel free to contact haejinso@usc.edu for any questions.
"""

In [ ]:
# Set up dashboards
explorer = FLDASExplorer()
widgets = pn.Param(explorer.param, widgets={
    'time': pn.widgets.DatePicker
})


In [ ]:
ze = ZonalExplorer()

In [ ]:
# Panel UI components
W_HEADER = 300 + W_IMG*2
texts = pn.Column( pn.Pane(title, width=W_HEADER),
          pn.Pane(instruction, width=W_HEADER))
viz_title = pn.Pane(viz_title)

tseries_title = pn.Pane(tseries_title)

zonal_title = pn.Pane(zonal_title)
zonal_text = pn.Pane(zonal_instruction, width=W_HEADER)

contact_text = pn.Pane(contact, width=W_HEADER)

In [ ]:
# 1. First dashboard
## main viz panel components
header = pn.Column(texts,viz_title)
viz_panel = pn.Column(explorer.view)
row1 = pn.Row(widgets, viz_panel)
dashboard1 = pn.Column(header, row1)#.servable()

In [ ]:
# 2. Second roi explorer dashboard
dashboard2 = pn.Column(
  zonal_title,
  zonal_text,
    pn.panel(ze.param),
    pn.panel(ze.panel_roi)
)#.servable()

In [ ]:
pn.Column(dashboard1, 
#           pn.layout.HSpacer(),

          dashboard2,
         contact_text).servable()

---
Modified: Jul 7, 2019

## Histogram-based visualizations

todo
- [ ] Click on a point and append the regional values as a curve
- [ ] colorbar informationn -- how to do it for `datashade` output?




In [ ]:
timg_ea.hist()

In [ ]:
timg_ea.data

In [ ]:
temp

In [ ]:
curve_at_x = hv.Curve(temp.sel(X=50, method='nearest'),'Y', varname).opts(padding=0.1)

In [ ]:
subset = temp.sel(X=50, method='nearest')

In [ ]:
subset


In [ ]:
gv.Image(timg_ea) * gv.VLine(50) # << curve_at_x << curve_at_x

In [ ]:
gv.Shape(LinkedStream)

In [ ]:
from shapely.geometry import LineString

In [ ]:
# Helper 
def get_minmax(dataset, dimname):
    vals = dataset.coords.get(dimname)
    return vals.min().item(), vals.max().item()

In [ ]:
data = timg_ea.data
xmin,xmax = get_minmax(data, 'X')
ymin, ymax = get_minmax(data, 'Y')

# Draw stright lines at mouse click
def line_at_x(x, yrange=None):
    if yrange is None:
        ymin, ymax = get_minmax(data, 'Y')
    else:
        ymin, ymax = yrange
        
    l = LineString([(x,ymin), (x,ymax)])
    return gv.Shape(l).opts(line_color='red', line_width=5)
    
def line_at_y(y, xrange=None):
    if xrange is None:
        xmin, xmax = get_minmax(data, 'X')
    else:
        xmin, xmax = xrange
        
    l = LineString([(xmin,y), (xmax,y)])
    return gv.Shape(l).opts(line_color='red', line_width=5)

# Get curve at x along the other dimention (ie. y), and vice versa for a fixed point Y=y
## depends on varname, data (at region, varname, time t)
def curve_at_x(data, x):
    return hv.Curve(data.sel(X=x, method='nearest'),'Y', varname, group='InvertedAxes').opts(padding=0.1)
def curve_at_y(data, y):
    return hv.Curve(data.sel(Y=y, method='nearest'),'X', varname).opts(padding=0.1)

In [ ]:
# test1
timg_ea * line_at_x(30).opts(line_color='black', line_dash='dashed') * line_at_y(10).opts(line_color='black', line_dash='dashed')

In [ ]:
# Define dmap for the stringt lines at mouse click location
dmap_geolines = hv.DynamicMap(lambda x,y: line_at_x(x) * line_at_y(y), streams=[xystream])
dmap_curve_at_x = hv.DynamicMap(lambda x,y: curve_at_x(data, x).opts(inverted_curve_opts), streams=[xystream])
dmap_curve_at_y = hv.DynamicMap(lambda x,y: curve_at_y(data, y), streams=[xystream])

In [ ]:
# Register stream on the image
xystream = Tap(x=35., y=0., source=timg_ea)
xystream.print_param_values()

In [ ]:
timg_ea

In [ ]:
dmap_curve_at_x

In [ ]:
(datashade(timg_ea) + dmap_curve_at_x).opts(shared_axes=False)


In [ ]:
# Putting them toegether
(
    datashade(timg_ea) * dmap_geolines 
    + dmap_curve_at_x.opts(curve_x_opts)
    + dmap_curve_at_y.opts()
    
).opts(shared_axes=False)

In [ ]:
%%opts Overlay(axiswise=True)
p1 = pn.panel(dmap_curve_at_y.opts(height=H_PLOT, width=W_IMG))
p2 = pn.panel( basemap * (datashade(timg_ea) * dmap_geolines).opts(height=H_IMG, width=W_IMG) )
p3 = pn.panel(dmap_curve_at_x.opts(height=H_IMG, width=H_PLOT))
pn.Column(p1,
         pn.Row(p2,p3))


In [ ]:
dmap_curve_at_y.opts.info()

In [ ]:
dmap_curve_at_x.opts.info()

In [ ]:
hv.help(hv.Curve)